In [5]:
%cd ../../src

e:\00_CODE\03_Master_Thesis\rdf-literal-preprocessing\src


In [6]:
import fire
import sys
import tqdm
import pandas as pd
import numpy as np
import kgbench as kg
from collections import Counter
import matplotlib.pyplot as pl
import random as rd
from operator import itemgetter
import torch
import torch.nn.functional as F
from utils import RDF_NUMBER_TYPES, get_relevant_relations, add_triple, get_p_types, ALL_LITERALS
from kgbench.load import Data
from typing import List, Sequence, Tuple

from kgbench import load, tic, toc, d
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
from utils import URI_PREFIX
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
import datetime

In [7]:
from dataload import amplus
data = amplus()

loaded data amplus (49.12s).
pruned (29.03s).


In [8]:
# REMAP to symmetric relations a <-> b
#maped_data = data.triples.copy
for t in data.triples:
    t[0] = torch.tensor(data.e2i[data.i2e[t[0]]], dtype=torch.int32)
    t[1] = torch.tensor(data.r2i[data.i2r[t[1]]], dtype=torch.int32)
    t[2] = torch.tensor(data.e2i[data.i2e[t[2]]], dtype=torch.int32)

for t in data.training:
    t[0] = torch.tensor(data.e2i[data.i2e[t[0]]])

for t in data.withheld:
    t[0] = torch.tensor(data.e2i[data.i2e[t[0]]])



C:\Users\Noctris\AppData\Local\Temp\ipykernel_25768\3451094937.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t[2] = torch.tensor(data.e2i[data.i2e[t[2]]], dtype=torch.int32)


In [9]:
base_e_unique = torch.unique(torch.cat([data.triples[:,0],data.triples[:,2]])) 
base_r_unique = torch.unique(data.triples[:,1])

In [10]:

new_e2i = {}
new_i2e = []

# TODO filter problem possibly here?!?! todo tomorrow
    
for i in range(len(data.i2e)):
    if i in base_e_unique.numpy():
        #print("here")
        new_e2i[data.i2e[i]] = len(new_i2e)
        new_i2e.append(data.i2e[i])

    #create new r mapping
new_r2i = {}
new_i2r = []

for i in range(len(data.i2r)):
    if i in base_r_unique.numpy():
        new_r2i[data.i2r[i]] = len(new_i2r)
        new_i2r.append(data.i2r[i])

    # apply new mapping for triples
#for t in filtered:
#    t[0] = new_e2i[data.i2e[t[0]]]
#    t[1] = new_r2i[data.i2r[t[1]]]
#    t[2] = new_e2i[data.i2e[t[2]]]

In [11]:
len(new_i2e)

825677

In [17]:
import pandas as pd
len(pd.unique(new_i2e))

825677

In [18]:
# REMAP to symmetric relations a <-> b
#maped_data = data.triples.copy
for t in data.triples:
    t[0] = torch.tensor(new_e2i[data.i2e[t[0]]], dtype=torch.int32)
    t[1] = torch.tensor(new_r2i[data.i2r[t[1]]], dtype=torch.int32)
    t[2] = torch.tensor(new_e2i[data.i2e[t[2]]], dtype=torch.int32)

for t in data.training:
    t[0] = torch.tensor(new_e2i[data.i2e[t[0]]])

for t in data.withheld:
    t[0] = torch.tensor(new_e2i[data.i2e[t[0]]])


In [19]:
    #update metedata
data.num_entities = len(new_i2e)
data.num_relations = len(new_i2r)

#     #update data
#data.triples = filtered
data.i2e = new_i2e
data.e2i = new_e2i
data.i2r = new_i2r
data.r2i = new_r2i
#data.training = torch.tensor(new_train)
#data.withheld = torch.tensor(new_withheld)
  

In [20]:
for r in range(len(data.i2e)):
    if data.i2e[r] == ('10', 'none'):
        print(data.e2i[data.i2e[r]])
        print("len")
        print(len(data.triples[data.triples[:,2]== torch.tensor(r, dtype=torch.int32)]))
        print(r)
    #break

960
len
22
960


In [21]:
from preprocess import bin_numbers_10
data = bin_numbers_10(data)

10
deleting relations [825684, 825685], since no occurences are given
done deleteing
unnamed_dataset


In [22]:
#from utils import delete_r
import torch
rr = get_relevant_relations(data,RDF_NUMBER_TYPES)

filtered = data.triples[~(torch.isin(data.triples[:,1],torch.Tensor(rr)))]
    # get neg e filter
base_e_unique = torch.unique(torch.cat([data.triples[:,0],data.triples[:,2]])) 
filtered_e_unique = torch.unique(torch.cat([filtered[:,0],filtered[:,2]])) 
neg_e_filter =  base_e_unique[~(torch.isin(base_e_unique, filtered_e_unique))]

#get neg r filter
base_r_unique = torch.unique(data.triples[:,1])
filtered_r_unique = torch.unique(filtered[:,1])
neg_r_filter =  base_r_unique[~(torch.isin(base_r_unique, filtered_r_unique))]

#create new e mapping
new_e2i = {}
new_i2e = []


    # TODO filter problem possibly here?!?! todo tomorrow
    
for i in range(len(data.i2e)):
    if i not in neg_e_filter.numpy():
        #print("here")
        new_e2i[data.i2e[i]] = len(new_i2e)
        new_i2e.append(data.i2e[i])
    #create new r mapping
new_r2i = {}
new_i2r = []

for i in range(len(data.i2r)):
    if i not in neg_r_filter.numpy():
        new_r2i[data.i2r[i]] = len(new_i2r)
        new_i2r.append(data.i2r[i])

    # apply new mapping for triples
for t in filtered:
    t[0] = new_e2i[data.i2e[t[0]]]
    t[1] = new_r2i[data.i2r[t[1]]]
    t[2] = new_e2i[data.i2e[t[2]]]
        #t[0] = torch.tensor(new_e2i[data.i2e[t[0].numpy()]], dtype=torch.int32)
        #t[1] = torch.tensor(new_r2i[data.i2r[t[1].numpy()]], dtype=torch.int32)
        #t[2] = torch.tensor(new_e2i[data.i2e[t[2].numpy()]], dtype=torch.int32)

    # create new train & withheld
new_train =  []
new_withheld = []

    # calculate new train & withheld
for ent in data.training:
    new_train.append([new_e2i[data.i2e[ent[0].numpy()]],ent[1]])
    
for ent in data.withheld:
    new_withheld.append([new_e2i[data.i2e[ent[0].numpy()]],ent[1]])

    #update metedata
data.num_entities = len(new_i2e)
data.num_relations = len(new_i2r)

#     #update data
data.triples = filtered
data.i2e = new_i2e
data.e2i = new_e2i
data.i2r = new_i2r
data.r2i = new_r2i
data.training = torch.tensor(new_train)
data.withheld = torch.tensor(new_withheld)
  
# return data
# data = delete_r(data, RDF_NUMBER_TYPES)

In [23]:
df =pd.DataFrame(data.triples, columns=["h","r","t"])
df['h'] = df['h'].apply(lambda h:data.i2e[h][0])
df['r'] = df['r'].apply(lambda r:data.i2r[r])
df['t'] = df['t'].apply(lambda t:data.i2e[t][0])

In [24]:
df.to_csv(f'hereeeee.tsv.gz', 
        index=False,
        sep="\t" ,
        compression="gzip")

In [55]:
data.triples[data.triples[:,2]== 852]

tensor([], size=(0, 3), dtype=torch.int64)

In [26]:
for e in data.i2e:
    if e==('10', 'none'):
        print(data.e2i[e])
        break

In [39]:
data.i2e[507017]

('10', 'none')

In [41]:
new_i2e[137231]

('_:N0f877559f6724858b2c9bd330a0a0a54', 'blank_node')

In [46]:
for t in data.triples[data.triples[:,0]==137231]:
    print(f'{data.i2e[t[0]]} - {data.i2r[t[1]]} - {data.i2e[t[2]]}')

('_:N09d973ef73704ccab8bd4a570e53cf65', 'blank_node') - http://purl.org/collections/nl/am/dimensionType - ('http://purl.org/collections/nl/am/t-16103', 'iri')
('_:N09d973ef73704ccab8bd4a570e53cf65', 'blank_node') - http://purl.org/collections/nl/am/dimensionUnit - ('0', 'none')
('_:N09d973ef73704ccab8bd4a570e53cf65', 'blank_node') - http://purl.org/collections/nl/am/dimensionValue - ('10', 'none')
('_:N09d973ef73704ccab8bd4a570e53cf65', 'blank_node') - http://www.w3.org/1999/02/22-rdf-syntax-ns#type - ('http://purl.org/collections/nl/am/Dimension', 'iri')
('_:N09d973ef73704ccab8bd4a570e53cf65', 'blank_node') - http://master-thesis.com/predicat#binning11 - ('http://master-thesis.com/entity#binning1', 'http://master-thesis.com/datatype#bin')


In [47]:
df[df['h']=='_:N09d973ef73704ccab8bd4a570e53cf65']

,h,r,t
32222,_:N09d973ef73704ccab8bd4a570e53cf65,http://purl.org/collections/nl/am/dimensionType,http://purl.org/collections/nl/am/t-16103
32223,_:N09d973ef73704ccab8bd4a570e53cf65,http://purl.org/collections/nl/am/dimensionUnit,0
32224,_:N09d973ef73704ccab8bd4a570e53cf65,http://purl.org/collections/nl/am/dimensionValue,10
32225,_:N09d973ef73704ccab8bd4a570e53cf65,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://purl.org/collections/nl/am/Dimension
2189702,_:N09d973ef73704ccab8bd4a570e53cf65,http://master-thesis.com/predicat#binning11,http://master-thesis.com/entity#binning1


In [48]:
data.i2r[11]

'http://purl.org/collections/nl/am/dimensionValue'

In [49]:
rr = get_relevant_relations(data, RDF_NUMBER_TYPES)

In [50]:
rr

[11, 15]